In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
def sparql(REQ):
    return list(default_world.sparql(REQ))

In [3]:
onto = get_ontology("WIP.owl").load()
dIDct = u.createDict(onto) 

PBNThing
BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Stakeholder_Type
Technology
ISO_Impact
ISO_Purpose
StakeholderGroup
StakeholderSubgroup
TechGroup
TechSubgroup
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
PBNCategory
RiskGroup
RiskSubgroup
CAO_Group
CAO_Subgroup
BP_Theme
BP_Category
aBlueprint


In [4]:
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


# 1. [x] Doing connections to Risks

### [x] RiskGroup -> TechGroup

In [5]:
with onto:
    class has_RiskGroup_TechGroup(onto.RiskGroup >> onto.TechGroup):
        label = ["Technology groups captured in this risk group"]
        pass
    class has_RiskGroup_TechSubgroup(onto.RiskGroup >> onto.TechSubgroup):
        label = ["Technology subgroups captured in this risk group"]
        pass
    class has_RiskSubgroup_TechGroup(onto.RiskSubgroup >> onto.TechGroup):
        label = ["Technology groups captured in this risk subgroup"]
        pass
    class has_RiskSubgroup_TechSubgroup(onto.RiskSubgroup >> onto.TechSubgroup):
        label = ["Technology subgroups captured in this risk subgroup"]
        pass

In [6]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechGroup.append(sTG)
        sRG.has_RiskGroup_TechGroup = list(set(sRG.has_RiskGroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskGroup a pbn:RiskGroup . 
    ?techSubgroup a pbn:TechSubgroup . 
            
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_TechSubgroup.append(sTG)
        sRG.has_RiskGroup_TechSubgroup = list(set(sRG.has_RiskGroup_TechSubgroup))


In [7]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techGroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techGroup a pbn:TechGroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechGroup.append(sTG)
        sRG.has_RiskSubgroup_TechGroup = list(set(sRG.has_RiskSubgroup_TechGroup))

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?techSubgroup (COUNT(?technology) AS ?techCount)
    WHERE {

    ?techSubgroup a pbn:TechSubgroup  .
    ?technology a pbn:Technology . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_TechSubgroup.append(sTG)
        sRG.has_RiskSubgroup_TechSubgroup = list(set(sRG.has_RiskSubgroup_TechSubgroup))


### [x] RiskGroups -> BP_Interventions

In [8]:
with onto:
    class has_RiskGroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        label = ["Inteventions captured in this risk group"]
        pass
    class has_RiskSubgroup_BPIntervention(onto.RiskGroup >> onto.BP_Intervention):
        label = ["Inteventions captured in this risk subgroup"]
        pass


In [9]:
for RG in [x.name for x in onto.RiskGroup.instances()][:-2]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskGroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskGroup a pbn:RiskGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskGroup ?riskGroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskGroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


In [10]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()][:-2]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?riskSubgroup ?interv  (COUNT(?mitig) AS ?mCount)
        WHERE {

        ?riskSubgroup a pbn:RiskSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        
        ?risk pbn:has_RiskSubgroup ?riskSubgroup  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .
        FILTER (?riskSubgroup = pbn:"""+RSG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_BPIntervention.append(sTG)
        sRG.has_RiskGroup_BPIntervention = list(set(sRG.has_RiskGroup_BPIntervention))


### [x] RiskGroups -> Owners

In [11]:
with onto:
    class has_RiskGroup_OwnerGroup(onto.RiskGroup >> onto.StakeholderGroup):
        label = ["People groups who can act on this risk group"]
        pass
    class has_RiskGroup_OwnerSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can act on this risk group"]
        pass
    class has_RiskSubgroup_OwnerGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        label = ["People groups who can act on this risk subgroup"]
        pass
    class has_RiskSubgroup_OwnerSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can act on this risk subgroup"]
        pass

In [12]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_OwnerGroup.append(sTG)
        sRG.has_RiskGroup_OwnerGroup = list(set(sRG.has_RiskGroup_OwnerGroup))

In [13]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_OwnerGroup.append(sTG)
        sRG.has_RiskSubgroup_OwnerGroup = list(set(sRG.has_RiskSubgroup_OwnerGroup))

### [x] RiskGroups -> Subject

In [14]:
with onto:
    class has_RiskGroup_SubjectGroup(onto.RiskGroup >> onto.StakeholderGroup):
        label = ["People groups who are subject to this risk group"]
        pass
    class has_RiskGroup_SubjectSubgroup(onto.RiskGroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who are subject to this risk group"]
        pass
    class has_RiskSubgroup_SubjectGroup(onto.RiskSubgroup >> onto.StakeholderGroup):
        label = ["People groups who are subject to this risk subgroup"]
        pass
    class has_RiskSubgroup_SubjectSubgroup(onto.RiskSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who are subject to this risk subgroup"]
        pass




In [15]:
for RG in [x.name for x in onto.RiskGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectGroup.append(sTG)
        sRG.has_RiskGroup_SubjectGroup = list(set(sRG.has_RiskGroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskGroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskGroup_SubjectSubgroup = list(set(sRG.has_RiskGroup_SubjectSubgroup))

In [16]:
for RSG in [x.name for x in onto.RiskSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectGroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectGroup = list(set(sRG.has_RiskSubgroup_SubjectGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?riskSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?riskSubgroup = pbn:"""+RSG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_RiskSubgroup_SubjectSubgroup.append(sTG)
        sRG.has_RiskSubgroup_SubjectSubgroup = list(set(sRG.has_RiskSubgroup_SubjectSubgroup))

# 2. Connections from Techs

##  [x] Techs -> Stakeholders

In [17]:
with onto:
    class has_TechGroup_StakeholderGroup(onto.TechGroup >> onto.StakeholderGroup):
        label = ["People groups who can use or are impacted by this technology group"]
        pass
    class has_TechGroup_StakeholderSubgroup(onto.TechGroup >> onto.StakeholderSubgroup):
        laebl = ["People subgroups who can use or are impacted by this technology group"]
        pass


In [18]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .

                    
    ?risk ?o ?sh .
               
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 

                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_StakeholderGroup.append(sTG)
        sRG.has_TechGroup_StakeholderGroup = list(set(sRG.has_TechGroup_StakeholderGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techGroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_StakeholderSubgroup.append(sTG)
        sRG.has_TechGroup_StakeholderSubgroup = list(set(sRG.has_TechGroup_StakeholderSubgroup))
    print(A)
#17s

PBN__TechGroup_0
[[WIP.PBN__TechGroup_0, WIP.PBN__StakeholderGroup_7, 5206], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderGroup_2, 5191], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderGroup_1, 4468]]
[[WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_3, 1769], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_15, 1594], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_5, 934], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_2, 651], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_63, 648], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_1, 619], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_14, 573], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_58, 465], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_10, 394], [WIP.PBN__TechGroup_0, WIP.PBN__StakeholderSubgroup_139, 384]]
PBN__TechGroup_1
[[WIP.PBN__TechGroup_1, WIP.PBN__StakeholderGroup_6, 5732], [WIP.PBN__TechGroup_1, WIP.PBN__StakeholderGroup_1, 5045], [WIP.PBN__TechGroup_1, WIP.PBN__StakeholderGroup_

In [19]:
with onto:
    class has_TechSubgroup_StakeholderGroup(onto.TechSubgroup >> onto.StakeholderGroup):
        label = ["People groups who can use or are impacted by this technology subgroup"]
        pass
    class has_TechSubgroup_StakeholderSubgroup(onto.TechSubgroup >> onto.StakeholderSubgroup):
        label = ["People subgroups who can use or are impacted by this technology subgroup"]
        pass


In [20]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techSubgroup ?shGroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_StakeholderGroup.append(sTG)
        sRG.has_TechSubgroup_StakeholderGroup = list(set(sRG.has_TechSubgroup_StakeholderGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?shSubgroup (COUNT(?sh) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?shSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_StakeholderSubgroup.append(sTG)
        sRG.has_TechSubgroup_StakeholderSubgroup = list(set(sRG.has_TechSubgroup_StakeholderSubgroup))
    print(A)
#3mins46 initially // 5.42

PBN__TechSubgroup_0
[[WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderGroup_7, 357], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderGroup_2, 321], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderGroup_0, 239]]
[[WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_15, 213], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_3, 143], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_2, 76], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_5, 71], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_63, 57], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_10, 55], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_1, 52], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_58, 51], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_40, 51], [WIP.PBN__TechSubgroup_0, WIP.PBN__StakeholderSubgroup_95, 30]]
PBN__TechSubgroup_1
[[WIP.PBN__TechSubgroup_1, WIP.PBN__StakeholderGroup_7, 485], [WIP.PBN__TechSubgroup_1, WIP.PBN__StakeholderGroup_1, 481], [WIP.PBN__Tech

##  [x] Techs -> Mitigations

In [21]:
with onto:
    class has_TechGroup_BPIntervention(onto.TechGroup >> onto.BP_Intervention):
        label = ["Interventions enabled by this technology group"]
        pass
    class has_TechSubgroup_BPIntervention(onto.TechSubgroup >> onto.BP_Intervention):
        label = ["Interventions enabled by this technology subgroup"]
        pass


In [22]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?interv (COUNT(?risk) AS ?sCount)
    WHERE {

    ?techGroup a pbn:TechGroup .
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
               
    ?technology pbn:has_TechGroup ?techGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                

                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?interv
    ORDER BY DESC(?sCount)
    LIMIT 3""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_BPIntervention.append(sTG)
        sRG.has_TechGroup_BPIntervention = list(set(sRG.has_TechGroup_BPIntervention))
    print(A)


PBN__TechGroup_0
[[WIP.PBN__TechGroup_0, WIP.PBN__BP_Intervention_29, 69215], [WIP.PBN__TechGroup_0, WIP.PBN__BP_Intervention_38, 67437], [WIP.PBN__TechGroup_0, WIP.PBN__BP_Intervention_37, 53975]]
PBN__TechGroup_1
[[WIP.PBN__TechGroup_1, WIP.PBN__BP_Intervention_38, 61595], [WIP.PBN__TechGroup_1, WIP.PBN__BP_Intervention_1, 58039], [WIP.PBN__TechGroup_1, WIP.PBN__BP_Intervention_24, 43815]]
PBN__TechGroup_2
[[WIP.PBN__TechGroup_2, WIP.PBN__BP_Intervention_38, 182499], [WIP.PBN__TechGroup_2, WIP.PBN__BP_Intervention_29, 177038], [WIP.PBN__TechGroup_2, WIP.PBN__BP_Intervention_30, 165735]]
PBN__TechGroup_12
[[WIP.PBN__TechGroup_12, WIP.PBN__BP_Intervention_1, 90170], [WIP.PBN__TechGroup_12, WIP.PBN__BP_Intervention_42, 71628], [WIP.PBN__TechGroup_12, WIP.PBN__BP_Intervention_5, 64389]]
PBN__TechGroup_7
[[WIP.PBN__TechGroup_7, WIP.PBN__BP_Intervention_1, 88392], [WIP.PBN__TechGroup_7, WIP.PBN__BP_Intervention_38, 36703], [WIP.PBN__TechGroup_7, WIP.PBN__BP_Intervention_33, 33909]]
PBN__Te

In [23]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?interv (COUNT(?risk) AS ?sCount)
    WHERE {

    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
    ?interv a pbn:BP_Intervention .

    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .                        
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .         
                                
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?interv
    ORDER BY DESC(?sCount)
    LIMIT 5""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_BPIntervention.append(sTG)
        sRG.has_TechSubgroup_BPIntervention = list(set(sRG.has_TechSubgroup_BPIntervention))
    print(A)
#3mins46 initially // 5.42

[[WIP.PBN__TechSubgroup_0, WIP.PBN__BP_Intervention_29, 1860], [WIP.PBN__TechSubgroup_0, WIP.PBN__BP_Intervention_1, 1665], [WIP.PBN__TechSubgroup_0, WIP.PBN__BP_Intervention_30, 1440], [WIP.PBN__TechSubgroup_0, WIP.PBN__BP_Intervention_38, 1425], [WIP.PBN__TechSubgroup_0, WIP.PBN__BP_Intervention_37, 885]]
[[WIP.PBN__TechSubgroup_1, WIP.PBN__BP_Intervention_29, 1380], [WIP.PBN__TechSubgroup_1, WIP.PBN__BP_Intervention_1, 1155], [WIP.PBN__TechSubgroup_1, WIP.PBN__BP_Intervention_38, 810], [WIP.PBN__TechSubgroup_1, WIP.PBN__BP_Intervention_17, 720], [WIP.PBN__TechSubgroup_1, WIP.PBN__BP_Intervention_40, 630]]
[[WIP.PBN__TechSubgroup_2, WIP.PBN__BP_Intervention_1, 1500], [WIP.PBN__TechSubgroup_2, WIP.PBN__BP_Intervention_42, 1230], [WIP.PBN__TechSubgroup_2, WIP.PBN__BP_Intervention_5, 1170], [WIP.PBN__TechSubgroup_2, WIP.PBN__BP_Intervention_9, 720], [WIP.PBN__TechSubgroup_2, WIP.PBN__BP_Intervention_2, 675]]
[[WIP.PBN__TechSubgroup_3, WIP.PBN__BP_Intervention_1, 1260], [WIP.PBN__TechSub

##  [x] Techs -> Risks

In [24]:
with onto:
    class has_TechGroup_RiskGroup(onto.TechGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated with this technology group"]
        pass
    class has_TechGroup_RiskSubgroup(onto.TechGroup >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated with this technology group"]
        pass
    class has_TechSubgroup_RiskGroup(onto.TechSubgroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated with this technology subgroup"]
        pass
    class has_TechSubgroup_RiskSubgroup(onto.TechSubgroup >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated with this technology subgroup"]
        pass


### TechGroup -> Risks

In [25]:
for RG in [x.name for x in onto.TechGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_RiskGroup.append(sTG)
        sRG.has_TechGroup_RiskGroup = list(set(sRG.has_TechGroup_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techGroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskSubgroup a pbn:RiskSubgroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
                              
    FILTER (?techGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechGroup_RiskSubgroup.append(sTG)
        sRG.has_TechGroup_RiskSubgroup = list(set(sRG.has_TechGroup_RiskSubgroup))
#3mins46 initially // 5.42

PBN__TechGroup_0
PBN__TechGroup_1
PBN__TechGroup_2
PBN__TechGroup_12
PBN__TechGroup_7
PBN__TechGroup_3
PBN__TechGroup_4
PBN__TechGroup_5
PBN__TechGroup_6
PBN__TechGroup_9
PBN__TechGroup_8
PBN__TechGroup_10
PBN__TechGroup_11
PBN__TechGroup_13
PBN__TechGroup_14


### TechSubgroup -> Risks

In [26]:
for RG in [x.name for x in onto.TechSubgroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT   ?techSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_RiskGroup.append(sTG)
        sRG.has_TechSubgroup_RiskGroup = list(set(sRG.has_TechSubgroup_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?techSubgroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {


    ?riskSubgroup a pbn:RiskSubgroup . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?techSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_TechSubgroup_RiskSubgroup.append(sTG)
        sRG.has_TechSubgroup_RiskSubgroup = list(set(sRG.has_TechSubgroup_RiskSubgroup))
#3mins46 initially // 5.42

PBN__TechSubgroup_0
PBN__TechSubgroup_1
PBN__TechSubgroup_2
PBN__TechSubgroup_3
PBN__TechSubgroup_4
PBN__TechSubgroup_5
PBN__TechSubgroup_6
PBN__TechSubgroup_7
PBN__TechSubgroup_8
PBN__TechSubgroup_9
PBN__TechSubgroup_10
PBN__TechSubgroup_11
PBN__TechSubgroup_12
PBN__TechSubgroup_13
PBN__TechSubgroup_14
PBN__TechSubgroup_15
PBN__TechSubgroup_16
PBN__TechSubgroup_17
PBN__TechSubgroup_72
PBN__TechSubgroup_18
PBN__TechSubgroup_19
PBN__TechSubgroup_25
PBN__TechSubgroup_20
PBN__TechSubgroup_21
PBN__TechSubgroup_22
PBN__TechSubgroup_23
PBN__TechSubgroup_73
PBN__TechSubgroup_24
PBN__TechSubgroup_26
PBN__TechSubgroup_27
PBN__TechSubgroup_28
PBN__TechSubgroup_29
PBN__TechSubgroup_30
PBN__TechSubgroup_31
PBN__TechSubgroup_32
PBN__TechSubgroup_33
PBN__TechSubgroup_34
PBN__TechSubgroup_35
PBN__TechSubgroup_36
PBN__TechSubgroup_37
PBN__TechSubgroup_38
PBN__TechSubgroup_39
PBN__TechSubgroup_86
PBN__TechSubgroup_57
PBN__TechSubgroup_114
PBN__TechSubgroup_74
PBN__TechSubgroup_103
PBN__TechSubgroup_124

# 3. Stakeholders

## [x] Stakeholders --> Risks

In [27]:
with onto:
    class has_StakeholderGroup_TechGroup(onto.StakeholderGroup >> onto.TechGroup):
        label = ["Technology groups that can be used this people group"]
        pass
    class has_StakeholderGroup_TechSubgroup(onto.StakeholderGroup >> onto.TechSubgroup):
        label = ["Technology subgroups that can be used this people group"]
        pass
    class has_StakeholderSubgroup_TechGroup(onto.StakeholderSubgroup >> onto.TechGroup):
        label = ["Technology groups that can be used this people subgroup"]
        pass
    class has_StakeholderSubgroup_TechSubgroup(onto.StakeholderSubgroup >> onto.TechSubgroup):
        label = ["Technology subgroups that can be used this people subgroup"]
        pass


In [28]:
with onto:
    class has_StakeholderGroup_RiskGroup_Owner(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskSubgroup_Owner(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskGroup_Subject(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderGroup_RiskSubgroup_Subject(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass


## [o] Stakeholder -> Techs

In [29]:
if 0:
    for RG in [x.name for x in onto.StakeholderGroup.instances()]:

        print(RG)
        A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT  ?shGroup ?techGroup (COUNT(?risk) AS ?sCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?sh a pbn:Stakeholder . 
        ?riskGroup a pbn:RiskGroup . 
        ?risk a pbn:Risk . 
        ?techGroup a pbn:TechGroup .
        ?techSubgroup a pbn:TechSubgroup .
                        
        ?risk pbn:has_RiskGroup ?riskGroup .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh pbn:has_StakeholderGroup ?shGroup . 
        ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
        ?risk pbn:has_RiskTechnology ?technology .
        ?technology pbn:has_TechGroup ?techGroup . 
        ?technology pbn:has_TechSubgroup ?techSubgroup . 
                                
        FILTER (?shGroup = pbn:"""+RG+""") 
        }
                
        GROUP BY ?techGroup
        ORDER BY DESC(?sCount)
        LIMIT 3""")
        for a in A:
            [sRG, sTG, c] = a
            sRG.has_StakeholderGroup_TechGroup.append(sTG)
            sRG.has_StakeholderGroup_TechGroup = list(set(sRG.has_StakeholderGroup_TechGroup))


        A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT  ?shGroup ?techSubgroup (COUNT(?risk) AS ?sCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?sh a pbn:Stakeholder . 
        ?riskGroup a pbn:RiskGroup . 
        ?risk a pbn:Risk . 
        ?techGroup a pbn:TechGroup .
        ?techSubgroup a pbn:TechSubgroup .
                                
        ?risk pbn:has_RiskGroup ?riskGroup .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh pbn:has_StakeholderGroup ?shGroup . 
        ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
        ?risk pbn:has_RiskTechnology ?technology .
        ?technology pbn:has_TechGroup ?techGroup . 
        ?technology pbn:has_TechSubgroup ?techSubgroup . 
                                
        FILTER (?shGroup = pbn:"""+RG+""") 
        }
                
        GROUP BY ?techSubgroup
        ORDER BY DESC(?sCount)
        LIMIT 10""")
        for a in A:
            [sRG, sTG, c] = a
            sRG.has_StakeholderGroup_TechSubgroup.append(sTG)
            sRG.has_StakeholderGroup_TechSubgroup = list(set(sRG.has_StakeholderGroup_TechSubgroup))
    #3mins46 initially // 5.42

### Stakeholder groups

In [30]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskGroup_Owner.append(sTG)
        sRG.has_StakeholderGroup_RiskGroup_Owner = list(set(sRG.has_StakeholderGroup_RiskGroup_Owner))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskSubgroup_Owner.append(sTG)
        sRG.has_StakeholderGroup_RiskSubgroup_Owner = list(set(sRG.has_StakeholderGroup_RiskSubgroup_Owner))
    
    ## Subject

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskGroup_Subject.append(sTG)
        sRG.has_StakeholderGroup_RiskGroup_Subject = list(set(sRG.has_StakeholderGroup_RiskGroup_Subject))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskSubject ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?StakeholderGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_RiskSubgroup_Subject.append(sTG)
        sRG.has_StakeholderGroup_RiskSubgroup_Subject = list(set(sRG.has_StakeholderGroup_RiskSubgroup_Subject))

### Stakeholder subgroups

In [31]:
with onto:
    class has_StakeholderSubgroup_RiskGroup_Owner(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskSubgroup_Owner(onto.StakeholderSubgroup>> onto.RiskGroup ):
        label = ["Risk subgroups that can be mitigated by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskGroup_Subject(onto.StakeholderGroup >> onto.RiskGroup):
        label = ["Risk groups that can be impacted by this people subgroup"]
        pass
    class has_StakeholderSubgroup_RiskSubgroup_Subject(onto.StakeholderSubgroup >> onto.RiskGroup):
        label = ["Risk subgroups that can be impacted by this people subgroup"]
        pass

In [32]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shSubgroup ?riskGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
         
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_RiskGroup_Owner.append(sTG)
        sRG.has_StakeholderSubgroup_RiskGroup_Owner = list(set(sRG.has_StakeholderSubgroup_RiskGroup_Owner))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?riskSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?riskSubgroup a pbn:RiskSubgroup . 
         
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_RiskSubgroup_Owner.append(sTG)
        sRG.has_StakeholderSubgroup_RiskSubgroup_Owner = list(set(sRG.has_StakeholderSubgroup_RiskSubgroup_Owner))
    print(A)

[[WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_12, 478], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_28, 313], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_49, 300], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_3, 264], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_21, 210], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_45, 183], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_69, 164], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_37, 162], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_33, 121], [WIP.PBN__StakeholderSubgroup_15, WIP.PBN__RiskSubgroup_1, 111]]
[[WIP.PBN__StakeholderSubgroup_0, WIP.PBN__RiskSubgroup_3, 19], [WIP.PBN__StakeholderSubgroup_0, WIP.PBN__RiskSubgroup_28, 9], [WIP.PBN__StakeholderSubgroup_0, WIP.PBN__RiskSubgroup_12, 8], [WIP.PBN__StakeholderSubgroup_0, WIP.PBN__RiskSubgroup_45, 7], [WIP.PBN__StakeholderSubgroup_0, WIP.PBN__RiskSubgroup_22, 7], [WIP.PBN__StakeholderSubg

## [x] Stakeholders --> Mitigations

In [33]:
with onto:
    class has_StakeholderGroup_BPIntervention(onto.StakeholderGroup >> onto.BP_Intervention):
        label = ["Interventions that can be lead by this people group"]
        pass
    class has_StakeholderSubgroup_BPIntervention(onto.StakeholderSubgroup >> onto.BP_Intervention):
        label = ["Interventions that can be lead by this people subgroup"]
        pass


In [34]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?shGroup ?interv  (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
               
        ?sh  pbn:has_StakeholderGroup ?shGroup .        
        ?risk pbn:has_RiskOwner ?sh .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .

        FILTER (?shGroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_BPIntervention.append(sTG)
        sRG.has_StakeholderGroup_BPIntervention = list(set(sRG.has_StakeholderGroup_BPIntervention))


In [35]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?shSubgroup ?interv  (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shSubgroup a pbn:StakeholderSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
        ?sh a pbn:Stakeholder  .
                       
        ?risk pbn:has_RiskMitigation ?mitig .
        ?risk pbn:has_RiskOwner ?sh .
        ?interv pbn:has_InterventionMitig ?mitig  .
        ?sh  pbn:has_StakeholderSubgroup ?shSubgroup .
               
        FILTER (?shSubgroup = pbn:"""+RG+""") 
        }
        
        GROUP BY ?interv
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_BPIntervention.append(sTG)
        sRG.has_StakeholderSubgroup_BPIntervention = list(set(sRG.has_StakeholderSubgroup_BPIntervention))


## [x] Stakeholders --> Techs

In [36]:
for RG in [x.name for x in onto.StakeholderGroup.instances()]:

    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?techGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
                    
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?shGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_TechGroup.append(sTG)
        sRG.has_StakeholderGroup_TechGroup = list(set(sRG.has_StakeholderGroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shGroup ?techSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shGroup a pbn:StakeholderGroup  .
    ?sh a pbn:Stakeholder . 
    ?riskGroup a pbn:RiskGroup . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
    ?techSubgroup a pbn:TechSubgroup .
                               
    ?risk pbn:has_RiskGroup ?riskGroup .
    ?risk pbn:has_RiskOwner ?sh .
    ?sh pbn:has_StakeholderGroup ?shGroup . 
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
                              
    FILTER (?shGroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderGroup_TechSubgroup.append(sTG)
        sRG.has_StakeholderGroup_TechSubgroup = list(set(sRG.has_StakeholderGroup_TechSubgroup))
#3mins46 initially // 5.42

PBN__StakeholderGroup_7
PBN__StakeholderGroup_0
PBN__StakeholderGroup_1
PBN__StakeholderGroup_8
PBN__StakeholderGroup_2
PBN__StakeholderGroup_6
PBN__StakeholderGroup_4
PBN__StakeholderGroup_3
PBN__StakeholderGroup_11
PBN__StakeholderGroup_5
PBN__StakeholderGroup_9
PBN__StakeholderGroup_10
PBN__StakeholderGroup_12
PBN__StakeholderGroup_13
PBN__StakeholderGroup_14
PBN__StakeholderGroup_15
PBN__StakeholderGroup_16
PBN__StakeholderGroup_17


In [37]:
for RG in [x.name for x in onto.StakeholderSubgroup.instances()]:
    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT  ?shSubgroup ?techGroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techGroup a pbn:TechGroup .
                               
    ?risk ?o ?sh .
    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechGroup ?techGroup . 
            
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?sCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_TechGroup.append(sTG)
        sRG.has_StakeholderSubgroup_TechGroup = list(set(sRG.has_StakeholderSubgroup_TechGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?shSubgroup ?techSubgroup (COUNT(?risk) AS ?sCount)
    WHERE {

    ?shSubgroup a pbn:StakeholderSubgroup  .
    ?sh a pbn:Stakeholder . 
    ?risk a pbn:Risk . 
    ?techSubgroup a pbn:TechSubgroup .                   


    ?sh pbn:has_StakeholderSubgroup ?shSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk ?o ?sh .
                           
    FILTER (?shSubgroup = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?sCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_StakeholderSubgroup_TechSubgroup.append(sTG)
        sRG.has_StakeholderSubgroup_TechSubgroup = list(set(sRG.has_StakeholderSubgroup_TechSubgroup))

PBN__StakeholderSubgroup_15
PBN__StakeholderSubgroup_0
PBN__StakeholderSubgroup_1
PBN__StakeholderSubgroup_2
PBN__StakeholderSubgroup_14
PBN__StakeholderSubgroup_3
PBN__StakeholderSubgroup_10
PBN__StakeholderSubgroup_55
PBN__StakeholderSubgroup_4
PBN__StakeholderSubgroup_5
PBN__StakeholderSubgroup_6
PBN__StakeholderSubgroup_7
PBN__StakeholderSubgroup_8
PBN__StakeholderSubgroup_139
PBN__StakeholderSubgroup_17
PBN__StakeholderSubgroup_9
PBN__StakeholderSubgroup_16
PBN__StakeholderSubgroup_11
PBN__StakeholderSubgroup_12
PBN__StakeholderSubgroup_13
PBN__StakeholderSubgroup_18
PBN__StakeholderSubgroup_19
PBN__StakeholderSubgroup_20
PBN__StakeholderSubgroup_21
PBN__StakeholderSubgroup_22
PBN__StakeholderSubgroup_23
PBN__StakeholderSubgroup_91
PBN__StakeholderSubgroup_24
PBN__StakeholderSubgroup_25
PBN__StakeholderSubgroup_26
PBN__StakeholderSubgroup_27
PBN__StakeholderSubgroup_28
PBN__StakeholderSubgroup_29
PBN__StakeholderSubgroup_30
PBN__StakeholderSubgroup_84
PBN__StakeholderSubgroup_31
P

# [x] 4. Mitigations

## [x] Mitigations --> Risks

In [38]:
with onto:
    class has_BPIntervention_RiskGroup(onto.BP_Intervention >> onto.RiskGroup):
        label = ["Risk groups that can be mitigated by this intervention"]
        pass
    class has_BPIntervention_RiskSubgroup(onto.BP_Intervention >> onto.RiskSubgroup):
        label = ["Risk subgroups that can be mitigated by this intervention"]
        pass


In [39]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?riskGroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
               

    ?interv pbn:has_InterventionMitig ?mitig  .    
    ?risk pbn:has_RiskMitigation ?mitig .                    
    ?risk pbn:has_RiskGroup ?riskGroup .

            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_RiskGroup.append(sTG)
        sRG.has_BPIntervention_RiskGroup = list(set(sRG.has_BPIntervention_RiskGroup))


    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?riskSubgroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskSubgroup a pbn:RiskSubgroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
                         
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                        
    ?risk pbn:has_RiskSubgroup ?riskSubgroup .
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?riskSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_RiskSubgroup.append(sTG)
        sRG.has_BPIntervention_RiskSubgroup = list(set(sRG.has_BPIntervention_RiskSubgroup))


## [x] Mitigations --> Techs

In [40]:
with onto:
    class has_BPIntervention_TechGroup(onto.BP_Intervention >> onto.TechGroup):
        label = ["Technology groups that can be used by this intervention"]
        pass
    class has_BPIntervention_TechSubgroup(onto.BP_Intervention >> onto.TechSubgroup):
        label = ["Technologysub groups that can be used by this intervention"]
        pass


In [41]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:
    print(RG)
    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?techGroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?riskGroup a pbn:RiskGroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .
               
    ?technology pbn:has_TechGroup ?techGroup . 
    ?risk pbn:has_RiskTechnology ?technology .
    ?interv pbn:has_InterventionMitig ?mitig  .    
    ?risk pbn:has_RiskMitigation ?mitig .                    
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techGroup
    ORDER BY DESC(?techCount)
    LIMIT 3""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_TechGroup.append(sTG)
        sRG.has_BPIntervention_TechGroup = list(set(sRG.has_BPIntervention_TechGroup))
    print(A)

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

    SELECT ?interv ?techSubgroup (COUNT(?risk) AS ?techCount)
    WHERE {

    ?techSubgroup a pbn:TechSubgroup  .
    ?interv a pbn:BP_Intervention  .
    ?risk a pbn:Risk .

    ?technology pbn:has_TechSubgroup ?techSubgroup . 
    ?risk pbn:has_RiskTechnology ?technology .                         
    ?risk pbn:has_RiskMitigation ?mitig .
    ?interv pbn:has_InterventionMitig ?mitig  .                        
            
    FILTER (?interv = pbn:"""+RG+""") 
    }
            
    GROUP BY ?techSubgroup
    ORDER BY DESC(?techCount)
    LIMIT 10""")
    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_TechSubgroup.append(sTG)
        sRG.has_BPIntervention_TechSubgroup = list(set(sRG.has_BPIntervention_TechSubgroup))
    print(A)

PBN__BP_Intervention_21
[]
[]
PBN__BP_Intervention_30
[[WIP.PBN__BP_Intervention_30, WIP.PBN__TechGroup_2, 11745], [WIP.PBN__BP_Intervention_30, WIP.PBN__Technology_1447, 3330], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechGroup_0, 3141]]
[[WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_7, 381], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_42, 288], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_19, 128], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_6, 109], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_66, 104], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_0, 96], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_14, 85], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_13, 73], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_92, 54], [WIP.PBN__BP_Intervention_30, WIP.PBN__TechSubgroup_57, 41]]
PBN__BP_Intervention_9
[[WIP.PBN__BP_Intervention_9, WIP.PBN__TechGroup_2, 8127], [WIP.PBN__BP_Intervention_9, WIP.PBN__TechGroup_12, 3510], [WIP.PBN_

## [x] Mitigations --> Stakeholders

In [42]:
with onto:
    class has_BPIntervention_StakeholderGroup(onto.BP_Intervention >> onto.StakeholderGroup):
        label = ["People groups that can be supported by this intervention"]
        pass
    class has_BPIntervention_StakeholderSubgroup(onto.BP_Intervention >> onto.StakeholderSubgroup):
        label = ["People subgroups that can be supported by this intervention"]
        pass


In [43]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?interv ?shGroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shGroup a pbn:StakeholderGroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
                
        ?interv pbn:has_InterventionMitig ?mitig  .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?risk pbn:has_RiskOwner ?sh .
        ?sh  pbn:has_StakeholderGroup ?shGroup .   
               
        FILTER (?interv = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shGroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_StakeholderGroup.append(sTG)
        sRG.has_BPIntervention_StakeholderGroup = list(set(sRG.has_BPIntervention_StakeholderGroup))


In [44]:
A

[[WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderGroup_1, 17],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderGroup_10, 15],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderGroup_4, 15],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderGroup_7, 15],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderGroup_2, 14]]

In [45]:
for RG in [x.name for x in onto.BP_Intervention.instances()]:

    A = sparql("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

        SELECT DISTINCT ?interv ?shSubgroup   (COUNT(?risk) AS ?mCount)
        WHERE {

        ?shSubgroup a pbn:StakeholderSubgroup  .
        ?risk a pbn:Risk  .
        ?mitig a pbn:Mitigation  .
        ?interv a pbn:BP_Intervention  .
               
        ?sh  pbn:has_StakeholderSubgroup ?shSubgroup .        
        ?risk pbn:has_RiskOwner ?sh .
        ?risk pbn:has_RiskMitigation ?mitig .
        ?interv pbn:has_InterventionMitig ?mitig  .

        FILTER (?interv = pbn:"""+RG+""") 
        }
        
        GROUP BY ?shSubgroup
        ORDER BY DESC(?mCount)           

        LIMIT 5""")

    for a in A:
        [sRG, sTG, c] = a
        sRG.has_BPIntervention_StakeholderSubgroup.append(sTG)
        sRG.has_BPIntervention_StakeholderSubgroup = list(set(sRG.has_BPIntervention_StakeholderSubgroup))


In [46]:
A

[[WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderSubgroup_3, 12],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderSubgroup_71, 7],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderSubgroup_15, 7],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderSubgroup_5, 5],
 [WIP.PBN__BP_Intervention_46, WIP.PBN__StakeholderSubgroup_20, 4]]

In [48]:
onto.save("./WIP_w_SPARQL.owl")